In [12]:
import collections
from nltk import ngrams
import numpy as np

def readFile(filePath):
    with open(filePath, "rb") as binary_file:
        data = binary_file.read()
    return data

def byteSequenceToNgrams(byteSequence, N):
    Ngrams = ngrams(byteSequence, N)
    return list(Ngrams)
    
def binaryFileToNgramCounts(sample, N):
    fileByteSequence = readFile(sample)
    fileNgrams = byteSequenceToNgrams(fileByteSequence, N)
    return collections.Counter(fileNgrams)

In [2]:
from os import listdir
from os.path import isfile, join
directories = ["Benign PE Samples", "Malicious PE Samples"]
N=2

In [3]:
NgramCountsAllFiles = collections.Counter([])
for datasetPath in directories:
    all_samples = [f for f in listdir(datasetPath) if isfile(join(datasetPath,f))]
    for sample in all_samples:
        filePath = join(datasetPath, sample)
        NgramCountsAllFiles += binaryFileToNgramCounts(filePath, N)

In [5]:
K1 = 1000
K1_most_frequent_Ngrams = NgramCountsAllFiles.most_common(K1)
K1_most_frequent_Ngrams_list = [x[0] for x in K1_most_frequent_Ngrams]

In [6]:
K1_most_frequent_Ngrams_list

[(0, 0),
 (255, 255),
 (72, 139),
 (1, 0),
 (204, 204),
 (0, 72),
 (0, 255),
 (72, 137),
 (72, 141),
 (2, 0),
 (72, 131),
 (255, 21),
 (144, 144),
 (0, 1),
 (4, 0),
 (0, 32),
 (32, 0),
 (3, 0),
 (64, 1),
 (255, 72),
 (68, 36),
 (0, 139),
 (64, 0),
 (133, 192),
 (101, 0),
 (254, 255),
 (255, 0),
 (72, 133),
 (0, 116),
 (6, 0),
 (73, 139),
 (5, 0),
 (131, 196),
 (8, 0),
 (0, 101),
 (131, 236),
 (0, 232),
 (92, 36),
 (0, 64),
 (32, 32),
 (0, 128),
 (7, 0),
 (0, 144),
 (116, 0),
 (137, 69),
 (97, 0),
 (108, 0),
 (76, 139),
 (255, 139),
 (76, 36),
 (0, 4),
 (192, 116),
 (139, 203),
 (144, 72),
 (204, 72),
 (111, 0),
 (114, 0),
 (0, 105),
 (9, 0),
 (0, 233),
 (0, 111),
 (105, 0),
 (0, 114),
 (110, 0),
 (139, 77),
 (0, 80),
 (10, 0),
 (137, 68),
 (0, 97),
 (0, 110),
 (8, 72),
 (76, 141),
 (0, 2),
 (0, 76),
 (65, 0),
 (115, 0),
 (36, 32),
 (139, 200),
 (139, 206),
 (139, 207),
 (15, 132),
 (0, 112),
 (0, 115),
 (253, 255),
 (139, 69),
 (139, 64),
 (141, 77),
 (0, 6),
 (0, 16),
 (143, 143),
 (1

In [7]:
def featurizeSample(sample, K1_most_frequent_Ngrams_list):
    K1 = len(K1_most_frequent_Ngrams_list)
    feature_vector = K1*[0]
    fileNgrams = binaryFileToNgramCounts(sample, N)
    for i in range(K1):
        feature_vector[i]=fileNgrams[K1_most_frequent_Ngrams_list[i]]
    return feature_vector

In [9]:
directoriesWithLabels = [("Benign PE Samples",0), ("Malicious PE Samples",1)]
X = []
y = []
for datasetPath, label in directoriesWithLabels:
    all_samples = [f for f in listdir(datasetPath) if isfile(join(datasetPath,f))]
    for sample in all_samples:
        fileNum +=1
        filePath = join(datasetPath, sample)
        X.append(featurizeSample(filePath, K1_most_frequent_Ngrams_list))
        y.append(label)

In [14]:
X = np.asarray(X)

In [15]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2
K2 = 10

In [18]:
X_top_K2_freq = X[:,:K2]

In [19]:
mi_selector = SelectKBest(mutual_info_classif, k=K2)
X_top_K2_mi = mi_selector.fit_transform(X, y)

In [20]:
chi2_selector = SelectKBest(chi2, k=K2)
X_top_K2_ch2 = chi2_selector.fit_transform(X, y)